In [ ]:
!pip install rebound

     |████████████████████████████████| 243 kB 5.1 MB/s 
  Created wheel for rebound: filename=rebound-3.18.1-cp37-cp37m-linux_x86_64.whl size=578036 sha256=ec155b1c0e744ab7957b9b765f6e830cee5edccbb3cb3e41337e79826bd4fffd
  Stored in directory: /root/.cache/pip/wheels/2b/84/1e/7a046ad848432ec306b56c031799c86fdc5d6d385045f7020a
Successfully built rebound


In [ ]:
import rebound
import numpy as np
import time
from rebound.interruptible_pool import InterruptiblePool

# Import matplotlib
import matplotlib; matplotlib.use("pdf")
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import multiprocessing
import warnings
import csv
import threading 
import os

#connect to local runtime and use my local cores
#see these intructions https://research.google.com/colaboratory/local-runtimes.html
#home = "/mnt/c/Users/satur/Dropbox/Marialis/exomoon_packing/Google_Colab/"
#os.chdir(home)
#print(os.getcwd())
#sys.exit(0)

def write2file(fname,f_str,head):
    lock.acquire() # thread blocks at this line until it can obtain lock
    if head == "head":
        f = open(fname,'w')
    else:
        f = open(fname,'a')
    f.write(f_str)
    f.close()
    lock.release()

def Orb2Cart(x,m,mp):
    #routine to quickly convert Orbital elements to Cartesian coordinates
    temp_sim = rebound.Simulation()
    temp_sim.integrator = "ias15"
    temp_sim.units = ('days', 'AU', 'Msun')
    temp_sim.add(m=m)
    temp_sim.add(m=mp,a=x[0],e=x[1],inc=np.radians(x[2]),omega=np.radians(x[3]),Omega=np.radians(x[4]),M=np.radians(x[5]))
    temp_ps = temp_sim.particles
    temp = [mp,temp_ps[1].x,temp_ps[1].y,temp_ps[1].z,temp_ps[1].vx,temp_ps[1].vy,temp_ps[1].vz]
    return temp

def change_ref(Pl_state,sat_state):
    for i in range(1,7):
        sat_state[i] += Pl_state[i]
    return sat_state

def get_MA(idx):
    gold_ratio = (1.+np.sqrt(5.))/2.
    return np.radians(float(idx)*360.*gold_ratio % 360.)

def get_semi(idx,beta,a_mo,M_i,M_p): # Semi major axis of the moons 
    #idx = moon idx (starts from 0)
    #beta = spacing parameter (number of mutual hill radii)
    #a_mo = innermost moon semimajor axis
    #M_i = moon mass
    #M_p = planet mass
    # Paper: https://www.dropbox.com/s/b2s36tu7ju15mdt/Quarles_2018_AJ_155_130.pdf?dl=0 
    M_tilde = M_p + idx*M_i # total mass enclosed (excluding the current particle)
    X_m = 0.5*(2.*M_i/(3.*M_tilde))**(1./3.) #Mutual R_H (Eq.1),Combine the mass portion into a single variable X_m
    return a_mo*((1.+beta*X_m)/(1.-beta*X_m))**idx # Eq.2

def get_dist(ps,idx1,idx2):
    dx = ps[idx1].x - ps[idx2].x
    dy = ps[idx1].y - ps[idx2].y
    dz = ps[idx1].z - ps[idx2].z
    return np.sqrt(dx**2+dy**2+dz**2)

def whfast_simulation(par):

    bm, em = par # unpack parameters

    am = get_semi(N_moons-1,bm,a1,M_moon,M_E)
    #print(am,bm,X)

    qm = am*(1.-em)
    if qm < a1:
      tm = np.sqrt(qm**3/M_E)
    else:
      tm = np.sqrt(a1**3/M_E)

    sim = rebound.Simulation()
    sim.units=['yr', 'AU', 'Msun']
    sim.dt = 0.05*tm
    
    #sim.integrator = 'ias15'
    #sim.ri_ias15.min_dt = 1e-2 * sim.dt
    sim.integrator = "whfast"
    sim.ri_whfast.corrector = 11
    
    sim.collision = "line"
    #sim.ri_whfast.safe_mode = 0
    sim.add(m=1.,hash="Sun") #Sun-mass star
    sim.add(m=M_E,a=1,e=0.0167,hash="Earth",r=R_E)   #Earth-mass planet
    for i in range(0,N_moons):
        am = get_semi(i,bm,a1,M_moon,M_E)
        if i == (N_moons-1):
            sim.add(m=M_moon, a = am, e = 0.0, r=radius_moon, primary=sim.particles["Earth"])
        else: 
            sim.add(m=M_moon, a = am, e = em, r=radius_moon, primary=sim.particles["Earth"])
    sim.move_to_com()

    ps = sim.particles
    #for i in range(0,N_moons+1):
    #    print(ps[i+1].x,ps[i+1].y,ps[i+1].z,ps[i+1].vx,ps[i+1].vy,ps[i+1].vz)

    sim.exit_max_distance = 2.  # some moons can escape quickly and be captured around the Sun
    #sim.exit_min_distance = 6.32559e-6
    sim.init_megno()

    counter = 0
    stop_run = False
    tmax = 10.
    while not stop_run:
        sim_t = counter*sim.dt
        try:
            sim.integrate(sim_t, exact_finish_time=0) # integrate to the nearest timestep for each output
            #to keep the timestep constant and preserve WHFast's symplectic nature      
        except rebound.Escape as error: #probably don't need to use the rebound.Escape anymore
            #print("Completed",par)
            write2file(filename,"%1.3f, %1.3f, %1.3f\n" % (bm,em,10.),'foot')
            stop_run = True # stop simulation due to escape
        except rebound.Collision as error:
            write2file(filename,"%1.3f, %1.3f, %1.3f\n" % (bm,em,30.),'foot')
            stop_run = True #stop simulation due to collision

        #Check for moon escape on every step and stop simulation early if moon goes past R_H
        for k in range(2,N_moons+2):
            if get_dist(ps,1,k) > R_H: 
                write2file(filename,"%1.3f, %1.3f, %1.3f\n" % (bm,em,20.),'foot')
                stop_run = True  # stop simulation due to fast escape, but capture by Sun
        counter += 1
        if sim.t >= tmax:
            megno = sim.calculate_megno()    
            write2file(filename,"%1.3f, %1.3f, %1.3f\n" % (bm,em,megno),'foot')
            stop_run = True #stop simulation, but moons survive up to tmax
    return

lock = threading.Lock()

#Setup the moon type (parameters)
moon_mass = "C" #pick the type of Moon here!!!!!
N_moons = 6 #number of moons

M_E = 3.0035e-6 #Earth mass in M_sun
R_E = 4.26352e-5 #Earth radius in AU
a_p = 1.
M_star = 1.0 #Star mass in M_sun
R_H = a_p*(M_E/(3.*M_star))**(1./3.)
if moon_mass == "L":
    M_moon = 1./81.*M_E #Luna moon
    rho_moon = 3.3
    radius_moon = 0.2725*R_E
elif moon_mass == "C":
    M_moon = 0.00015*M_E #Ceres mass
    rho_moon = 2.08
    radius_moon = (476./6371)*R_E
elif moon_mass == "P":
    M_moon = 0.0022*M_E #Pluto mass
    rho_moon = 1.88
    radius_moon = 0.186*R_E

filename = 'megno_whfast_%s.csv' % moon_mass

#Setup the IC grid
#Ngrid = 100
a_min, a_max, a_step = 3.5, 9.5, 0.05
e_min, e_max, e_step = 0.0, 0.5, 0.005
#a_step = np.round((a_max-a_min)/Ngrid,3)
#e_step = np.round((e_max-e_min)/Ngrid,3)
par_a = np.arange(a_min,a_max+a_step,a_step)
par_e = np.arange(e_min,e_max+e_step,e_step)
#print(par_a)
#print(par_e)
parameters = []
for e in par_e:
    for a in par_a:
        parameters.append((a,e))

#os.remove(filename)
if not os.path.exists(filename): #check if output file already exists
    write2file(filename,'#beta,ecc,megno\n','head') #start from beginning
else:
    parameters = []
    eps = 1e-6
    done = np.genfromtxt(filename,delimiter=',',comments='#')
    if len(done.shape)>1:
        for e in par_e:
            for a in par_a:
                done_idx = np.where(np.logical_and(np.abs(a-done[:,0])<eps,np.abs(e-done[:,1])<eps))[0]
                if len(done_idx) == 0:
                    parameters.append((a,e)) #IC is not done add to list
                #otherwise IC is done, don't do anythong
print("number of ICs to run: ",len(parameters))

#Constants 
X = 0.5*(2.*M_moon/(3.*M_E))**(1./3.) #Mass term for R_{H,m}
R_R = 2.44*R_E*(5.515/rho_moon)**(1./3.)  #Roche limit using a fluid satellite defn
a1 = 2.*R_R # twice the Roche radius

t1=time.time()
#val_megno=whfast_simulation((4,0.1)) 
#val_megno=whfast_simulation((0.00001,0.0001))
t2=time.time()
#print("Timer",t2-t1)
#print(val_megno)

#whfast_simulation((4,0.1))

pool = InterruptiblePool()
t1=time.time()
#results = pool.map(whfast_simulation,parameters)

#Alternative to pool.map where the list of parameters is sampled more randomly
p_list = np.arange(0,len(parameters),dtype='i8')
np.random.shuffle(p_list)
jobs = []
for p in p_list:
    param = parameters[p]
    job = pool.apply_async(whfast_simulation, (param,))
    jobs.append(job)


for j in xrange(0,len(jobs)): 
    job.get()
t2=time.time()
#print("Timer:",(t2-t1))

number of ICs to run:  12322


In [ ]:
from scipy.interpolate import griddata
from matplotlib import rcParams

rcParams.update({'font.size': 22})
rcParams.update({'mathtext.fontset': 'cm'})

filename = 'megno_whfast_%s.csv' % moon_mass

a_min, a_max, a_step = 3.5, 9.5, 0.05
e_min, e_max, e_step = 0.0, 0.5, 0.005
par_a = np.arange(a_min,a_max+a_step,a_step)
par_e = np.arange(e_min,e_max+e_step,e_step)

X,Y,Z = np.genfromtxt(filename,delimiter=',',comments='#',unpack=True)

results2d = griddata((X,Y),Z,(par_a[None,:],par_e[:,None]),method='linear')
#results2d = results[:,2].reshape(Ngrid+1,Ngrid+1)

fs = 'x-large'
aspect = 4./3.
width = 7.

fig = plt.figure(figsize=(aspect*width,width))
ax = plt.subplot(111)
extent = [min(par_a),max(par_a),min(par_e),max(par_e)]

#im = ax.imshow(results2d, interpolation="nearest", vmin=2, vmax=8, cmap="RdYlGn_r", origin="lower", aspect='auto', extent=extent)
im = ax.pcolormesh(par_a,par_e,results2d,vmin=2, vmax=8, cmap="RdYlGn_r",shading='auto')
cb = plt.colorbar(im, ax=ax)
cb.set_label("MEGNO $\\langle Y \\rangle$", fontsize=fs)
cb.ax.tick_params(direction='out',length = 8.0, width = 6.0,labelsize=fs)

ax.set_xlabel("Initial Beta Spacing", fontsize=fs)
ax.set_ylabel("Initial Eccentricity ($e_o$)", fontsize=fs)

ax.minorticks_on()
ax.tick_params(which='major',axis='both', direction='out',length = 12.0, width = 8.0,labelsize=fs)
ax.tick_params(which='minor',axis='both', direction='out',length = 8.0, width = 6.0)

ax.set_ylim(0.0,0.5)
ax.set_xlim(3.5,9.5)

#ax.set_aspect('equal')

if moon_mass == 'C':
    fname = "CERES.png"
elif moon_mass == 'P':
    fname = "PLUTO.png"
else:
    fname = "LUNA.png"
fig.savefig(fname,bbox_inches = 'tight')
plt.close()

NameError: ignored